In [122]:
import numpy as np
import pyvista as pv
from matplotlib.cm import get_cmap
import astropy.io.fits as pyfits
import matplotlib

In [123]:
def norm(x, perc):
    x = (x - np.nanpercentile(x, 100-perc)) / (np.nanpercentile(x, perc) - np.nanpercentile(x, 100-perc))
    x[x < 0] = 0
    x[x > 1] = 1
    return x

def show_cloud(p, path, perc):
    _y, _z, _x, _scalar = pyfits.open(path)[0].data.T
        
    _scalar = norm(_scalar, perc)
    
    nonans = ~np.isnan(_scalar)  
    _scalar = _scalar[nonans]
    
    cloud = np.array([_x[nonans], _y[nonans], _z[nonans]]).T
    cloud = pv.PolyData(cloud)
    
    p.add_mesh(cloud, scalars=np.copy(_scalar), smooth_shading=False, opacity=.9)


def add_image(p, path, perc=99, binning=10):
    """
    :param pixsize: pizel size in arcsec
    """
    D_M1 = 2e3 # pc
    
    
    
  
    im = pyfits.open(path)[0]
    wcs = pywcs.WCS(im)
    xs, ys = pywcs.utils.proj_plane_pixel_scales(wcs)
    pixsize = np.mean((xs, ys)) * 3600
    scale = D_M1 * np.tan(np.deg2rad(1/3600.)) * pixsize # pc / pixel
    print('computed image scale (arcsec/pixels)', pixsize)
    yc, xc = wcs.all_world2pix((center_radec, ), 0)[0]
    
    print('computed center (pixels)', xc, yc)
    im_data = im.data.T
    print('image shape (pixels)', im_data.shape)
    
    im_data = norm(im_data[::binning,::binning], perc)
 
    
    # create a structured surface
    x = (np.arange(im_data.shape[0], dtype=float) - xc/binning) * scale * binning
    y = (np.arange(im_data.shape[1], dtype=float) - yc/binning) * scale * binning
    x, y = np.meshgrid(y, x)
    z = np.zeros_like(x)
    
    plane = pv.StructuredGrid(z, y, x)
    
    p.add_mesh(plane, scalars=im_data.T.flatten())

In [124]:
p = pv.PlotterITK()

#show_cloud(p, '3dmap_XYZvel.fits', 99)
show_cloud(p, '3dmap_XYZflux.fits', 90)

add_image(p, 'm1.deep_frame.fits')
p.show(True)


computed image scale (arcsec/pixels) 0.3203488372093011
computed center (pixels) 1076.9893810371195 1039.0079482857989
image shape (pixels) (2048, 2064)


Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…